In [1]:
! ls ../input/severstalmodels

128x128.pth	       unet_mobilenet2.pth  unet_se_resnext50_32x4d.pth
resnet34_classify.pth  unet_resnet34.pth


In [2]:
# ! python ../input/mlcomp/mlcomp/mlcomp/setup.py

In [3]:
import warnings
warnings.filterwarnings('ignore')
import os
import matplotlib.pyplot as plt

import numpy as np
import cv2
import albumentations as A
from tqdm import tqdm_notebook,tqdm
import pandas as pd

import torch
from torch.utils.data import DataLoader, Dataset
from torch.jit import load
import ast
from os.path import join
from numbers import Number
from collections import defaultdict
from typing import Callable, Dict

import tifffile

from torch.utils.data import Dataset

# from mlcomp.contrib.transform.albumentations import ChannelTranspose
# from mlcomp.contrib.dataset.classify import ImageDataset
# from mlcomp.contrib.transform.rle import rle2mask, mask2rle
# from mlcomp.contrib.transform.tta import TtaWrap

In [4]:
import sys
# Get necessary Imports
import pdb
import torch.backends.cudnn as cudnn
import albumentations as albu
from albumentations import (Normalize, Compose)
from albumentations.pytorch import ToTensor
import torch.utils.data as data
import torchvision.models as models
from torch import nn
import torch.nn.functional as F
from albumentations import ImageOnlyTransform

In [5]:
#https://github.com/catalyst-team/mlcomp/blob/master/mlcomp/contrib/torch/tensors.py
def flip(x, dim):
    indices = [slice(None)] * x.dim()
    indices[dim] = torch.arange(x.size(dim) - 1, -1, -1,
                                dtype=torch.long, device=x.device)
    return x[tuple(indices)]

In [6]:
#https://github.com/catalyst-team/mlcomp/blob/master/mlcomp/contrib/transform/albumentations.py
class ChannelTranspose(ImageOnlyTransform):
    def get_transform_init_args_names(self):
        return ()

    def get_params_dependent_on_targets(self, params):
        pass

    def __init__(self, axes=(2, 0, 1)):
        super().__init__(always_apply=True)
        self.axes = axes

    def apply(self, img, **params):
        return np.transpose(img, self.axes)

In [7]:
#https://github.com/catalyst-team/mlcomp/blob/master/mlcomp/contrib/transform/tta.py
class TtaWrap(Dataset):
    def __init__(self, dataset: Dataset, tfms=()):
        self.dataset = dataset
        self.tfms = tfms

    def __getitem__(self, item):
        return self.dataset[item]

    def __len__(self):
        return len(self.dataset)

    def inverse(self, a: np.array):
        last_dim = len(a.shape) - 1
        for t in self.tfms:
            if isinstance(t, A.HorizontalFlip):
                a = flip(a, last_dim)
            elif isinstance(t, A.VerticalFlip):
                a = flip(a, last_dim - 1)
            elif isinstance(t, A.Transpose):
                axis = (0, 1, 3, 2) if len(a.shape) == 4 else (0, 2, 1)
                a = a.permute(*axis)

        return a

In [8]:
class ImageDataset(Dataset):
    def __init__(
            self,
            *,
            img_folder: str,
            fold_csv: str = None,
            fold_number: int = None,
            is_test: bool = False,
            gray_scale: bool = False,
            num_classes=2,
            max_count=None,
            meta_cols=(),
            transforms=None,
            postprocess_func: Callable[[Dict], Dict] = None,
            include_image_orig=False
    ):
        self.img_folder = img_folder

        if fold_csv:
            df = pd.read_csv(fold_csv)
            if fold_number is not None:
                if is_test:
                    self.data = df[df['fold'] == fold_number]
                else:
                    self.data = df[df['fold'] != fold_number]
            else:
                self.data = df
        else:
            self.data = pd.DataFrame(
                {'image': os.listdir(img_folder)}).sort_values(by='image')

        self.data = self.data.to_dict(orient='row')
        if max_count is not None:
            self.apply_max_count(max_count)

        for row in self.data:
            self.preprocess_row(row)

        self.transforms = transforms
        self.gray_scale = gray_scale
        self.num_classes = num_classes
        self.meta_cols = meta_cols
        self.postprocess_func = postprocess_func
        self.include_image_orig = include_image_orig

    def apply_max_count(self, max_count):
        if isinstance(max_count, Number):
            self.data = self.data[:max_count]
        else:
            data = defaultdict(list)
            for row in self.data:
                data[row['label']].append(row)
            min_index = np.argmin(max_count)
            min_count = len(data[min_index])
            for k, v in data.items():
                count = int(min_count * (max_count[k] / max_count[min_index]))
                data[k] = data[k][:count]

            self.data = [v for i in range(len(data)) for v in data[i]]

    def preprocess_row(self, row: dict):
        row['image'] = join(self.img_folder, row['image'])

    def __len__(self):
        return len(self.data)

    def _get_item_before_transform(self, row: dict, item: dict):
        pass

    def _get_item_after_transform(self, row: dict,
                                  transformed: dict,
                                  res: dict):
        if 'label' in row:
            res['targets'] = ast.literal_eval(str(row['label']))
            if isinstance(res['targets'], list):
                res['targets'] = np.array(res['targets'], dtype=np.float32)

    def __getitem__(self, index):
        row = self.data[index]
        image = self.read_image_file(row['image'], self.gray_scale)
        item = {'image': image}

        self._get_item_before_transform(row, item)

        if self.transforms:
            item = self.transforms(**item)
        if self.gray_scale:
            item['image'] = np.expand_dims(item['image'], axis=0)
        res = {
            'features': item['image'].astype(np.float32),
            'image_file': row['image']
        }
        if self.include_image_orig:
            res['image'] = image

        for c in self.meta_cols:
            res[c] = row[c]

        self._get_item_after_transform(row, item, res)
        if self.postprocess_func:
            res = self.postprocess_func(res)
        return res

    @staticmethod
    def read_image_file(path: str, gray_scale=False):
        if path.endswith('.tiff') and not gray_scale:
            return tifffile.imread(path)
        elif path.endswith('.npy'):
            return np.load(path)
        else:
            if gray_scale:
                img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                assert img is not None, \
                    f'Image at path {path} does not exist'
                return img.astype(np.uint8)
            else:
                img = cv2.imread(path)
                assert img is not None, \
                    f'Image at path {path} does not exist'
                return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [9]:
unet_se_resnext50_32x4d = \
    load('/kaggle/input/severstalmodels/unet_se_resnext50_32x4d.pth').cuda()
unet_mobilenet2 = load('/kaggle/input/severstalmodels/unet_mobilenet2.pth').cuda()
unet_resnet34 = load('/kaggle/input/severstalmodels/unet_resnet34.pth').cuda()

In [10]:
class Model:
    def __init__(self, models):
        self.models = models
    
    def __call__(self, x):
        res = []
        x = x.cuda()
        with torch.no_grad():
            for m in self.models:
                res.append(m(x))
#         res = torch.stack(res)
#         return torch.mean(res, dim=0)
        return res

model = Model([unet_se_resnext50_32x4d, unet_mobilenet2, unet_resnet34])

In [11]:
def create_transforms(additional):
    res = list(additional)
    # add necessary transformations
    res.extend([
        A.Normalize(
            mean=(0.486, 0.457, 0.405), std=(0.230, 0.225, 0.224)
        ),
        ChannelTranspose()
    ])
    res = A.Compose(res)
    return res

img_folder = '/kaggle/input/severstal-steel-defect-detection/test_images'
batch_size = 16
num_workers = 0

# Different transforms for TTA wrapper
transforms = [
    [],
    [A.HorizontalFlip(p=1)]
#     [A.RandomCrop(256,800,p=1)]
]

transforms = [create_transforms(t) for t in transforms]
datasets = [TtaWrap(ImageDataset(img_folder=img_folder, transforms=t), tfms=t) for t in transforms]
loaders = [DataLoader(d, num_workers=num_workers, batch_size=batch_size, shuffle=False) for d in datasets]

In [12]:
import sys
MYLIB = '../input/ssdefectlib/'
WEIGHTS = '../input/ssdefectweights/'

In [13]:
from tqdm import tqdm_notebook,tqdm
sys.path.append(MYLIB)
from efficientnet import EfficientNetB5,BatchNorm2d,IMAGE_RGB_MEAN,IMAGE_RGB_STD,IS_GATHER_EXCITE,IS_PYTORCH_PAD
import albumentations as albu
from albumentations.pytorch import ToTensorV2

In [14]:
class ConvBn2d(nn.Module):

    def __init__(self, in_channel, out_channel, kernel_size=3, padding=1, stride=1):
        super(ConvBn2d, self).__init__()
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size, padding=padding, stride=stride, bias=False)
        self.bn   = nn.BatchNorm2d(out_channel, eps=1e-5)

    def forward(self,x):
        x = self.conv(x)
        x = self.bn(x)
        return x

class BasicBlock(nn.Module):
    def __init__(self, in_channel, channel, out_channel, stride=1, is_shortcut=False):
        super(BasicBlock, self).__init__()
        self.is_shortcut = is_shortcut

        self.conv_bn1 = ConvBn2d(in_channel,    channel, kernel_size=3, padding=1, stride=stride)
        self.conv_bn2 = ConvBn2d(   channel,out_channel, kernel_size=3, padding=1, stride=1)

        if is_shortcut:
            self.shortcut = ConvBn2d(in_channel, out_channel, kernel_size=1, padding=0, stride=stride)


    def forward(self, x):
        z = F.relu(self.conv_bn1(x),inplace=True)
        z = self.conv_bn2(z)

        if self.is_shortcut:
            x = self.shortcut(x)

        z += x
        z = F.relu(z,inplace=True)
        return z

class ResNet34(nn.Module):

    def __init__(self, num_class=1000 ):
        super(ResNet34, self).__init__()


        self.block0  = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, padding=3, stride=2, bias=False),
            BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.block1  = nn.Sequential(
             nn.MaxPool2d(kernel_size=3, padding=1, stride=2),
             BasicBlock( 64, 64, 64, stride=1, is_shortcut=False,),
          * [BasicBlock( 64, 64, 64, stride=1, is_shortcut=False,) for i in range(1,3)],
        )
        self.block2  = nn.Sequential(
             BasicBlock( 64,128,128, stride=2, is_shortcut=True, ),
          * [BasicBlock(128,128,128, stride=1, is_shortcut=False,) for i in range(1,4)],
        )
        self.block3  = nn.Sequential(
             BasicBlock(128,256,256, stride=2, is_shortcut=True, ),
          * [BasicBlock(256,256,256, stride=1, is_shortcut=False,) for i in range(1,6)],
        )
        self.block4 = nn.Sequential(
             BasicBlock(256,512,512, stride=2, is_shortcut=True, ),
          * [BasicBlock(512,512,512, stride=1, is_shortcut=False,) for i in range(1,3)],
        )
        self.logit = nn.Linear(512,num_class)



    def forward(self, x):
        batch_size = len(x)

        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        logit = self.logit(x)
        return logit
    
class Resnet34_classification(nn.Module):
    def __init__(self,num_class=4):
        super(Resnet34_classification, self).__init__()
        e = ResNet34()
        self.block = nn.ModuleList([
            e.block0,
            e.block1,
            e.block2,
            e.block3,
            e.block4,
        ])
        e = None  #dropped
        self.feature = nn.Conv2d(512,32, kernel_size=1) #dummy conv for dim reduction
        self.logit = nn.Conv2d(32,num_class, kernel_size=1)

    def forward(self, x):
        batch_size,C,H,W = x.shape

        for i in range( len(self.block)):
            x = self.block[i](x)
            #print(i, x.shape)

        x = F.dropout(x,0.5,training=self.training)
        x = F.adaptive_avg_pool2d(x, 1)
        x = self.feature(x)
        logit = self.logit(x)
        return logit

def getBibekClassifier():
    model_classification = Resnet34_classification()
    model_classification.load_state_dict(torch.load(WEIGHTS+'00007500_model.pth'))
    model_classification = model_classification.cuda()
    return model_classification

In [15]:
class ConvBn2d(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, dilation=1, stride=1, groups=1, is_bn=True):
        super(ConvBn2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride, dilation=dilation, groups=groups, bias=False)
        self.bn   = nn.BatchNorm2d(out_channels, eps=1e-5)

    def forward(self,x):
        x = self.conv(x)
        x = self.bn(x)
        return x
class SqueezeExcite(nn.Module):
    def __init__(self, in_channel, reduction=4, excite_size=-1):
        super(SqueezeExcite, self).__init__()
        self.excite_size=excite_size

        self.fc1 = nn.Conv2d(in_channel, in_channel//reduction, kernel_size=1, padding=0)
        self.fc2 = nn.Conv2d(in_channel//reduction, in_channel, kernel_size=1, padding=0)

    def forward(self, x):
        #print(x.shape)

        if IS_GATHER_EXCITE:
            #print('IS_GATHER_EXCITE')
            s = F.avg_pool2d(x, kernel_size=self.excite_size)
        else:
            s = F.adaptive_avg_pool2d(x,1)

        s = self.fc1(s)
        s = F.relu(s, inplace=True)
        s = self.fc2(s)

        if IS_GATHER_EXCITE:
            s = F.interpolate(s, size=(x.shape[2],x.shape[3]), mode='nearest')

        x = x*torch.sigmoid(s)
        return x


# bottleneck type C
class SENextBottleneckBlock(nn.Module):
    def __init__(self, in_channel, channel, out_channel, stride=1, group=32, reduction=16, excite_size=-1, is_shortcut=False):
        super(SENextBottleneckBlock, self).__init__()
        self.is_shortcut = is_shortcut

        self.conv_bn1 = ConvBn2d(in_channel,     channel, kernel_size=1, padding=0, stride=1)
        self.conv_bn2 = ConvBn2d(   channel,     channel, kernel_size=3, padding=1, stride=stride, groups=group)
        self.conv_bn3 = ConvBn2d(   channel, out_channel, kernel_size=1, padding=0, stride=1)
        self.scale    = SqueezeExcite(out_channel, reduction, excite_size)

        if is_shortcut:
            self.shortcut = ConvBn2d(in_channel, out_channel, kernel_size=1, padding=0, stride=stride)


    def forward(self, x):
        z = F.relu(self.conv_bn1(x),inplace=True)
        z = F.relu(self.conv_bn2(z),inplace=True)
        z = self.scale(self.conv_bn3(z))

        if self.is_shortcut:
            z += self.shortcut(x)
        else:
            z += x

        z = F.relu(z,inplace=True)
        return z

#resnext50_32x4d
class ResNext50(nn.Module):

    def __init__(self, num_class=1000 ):
        super(ResNext50, self).__init__()


        self.block0  = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )


        self.block1  = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, padding=0, stride=2, ceil_mode=True),
             SENextBottleneckBlock( 64, 128, 256, stride=1, is_shortcut=True,  excite_size=64),
          * [SENextBottleneckBlock(256, 128, 256, stride=1, is_shortcut=False, excite_size=64) for i in range(1,3)],
        )

        self.block2  = nn.Sequential(
             SENextBottleneckBlock(256, 256, 512, stride=2, is_shortcut=True,  excite_size=32),
          * [SENextBottleneckBlock(512, 256, 512, stride=1, is_shortcut=False, excite_size=32) for i in range(1,4)],
        )

        self.block3  = nn.Sequential(
             SENextBottleneckBlock( 512,512,1024, stride=2, is_shortcut=True,  excite_size=16),
          * [SENextBottleneckBlock(1024,512,1024, stride=1, is_shortcut=False, excite_size=16) for i in range(1,6)],
        )

        self.block4 = nn.Sequential(
             SENextBottleneckBlock(1024,1024,2048, stride=2, is_shortcut=True,  excite_size=8),
          * [SENextBottleneckBlock(2048,1024,2048, stride=1, is_shortcut=False, excite_size=8) for i in range(1,3)],
        )


        self.logit = nn.Linear(2048,num_class)



    def forward(self, x):
        batch_size = len(x)

        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        logit = self.logit(x)
        return logit

class Net2(nn.Module):
    def __init__(self, num_class=4):
        super(Net2, self).__init__()

        e = ResNext50()
        self.block0 = e.block0
        self.block1 = e.block1
        self.block2 = e.block2
        self.block3 = e.block3
        self.block4 = e.block4
        e = None  #dropped

        self.feature = nn.Conv2d(2048, 64, kernel_size=1) #dummy conv for dim reduction
        self.logit   = nn.Conv2d(64, num_class, kernel_size=1)



    def forward(self, x):
        batch_size,C,H,W = x.shape
        x = x.clone()
        x = x-torch.FloatTensor(IMAGE_RGB_MEAN).to(x.device).view(1,-1,1,1)
        x = x/torch.FloatTensor(IMAGE_RGB_STD).to(x.device).view(1,-1,1,1)

        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)

        x = F.dropout(x,0.5,training=self.training)
#         x = F.avg_pool2d(x, kernel_size=(8, 13),stride=(8, 8))
        x = F.adaptive_avg_pool2d(x, 1)
        x = self.feature(x)

        logit = self.logit(x) #.view(batch_size,-1)
        return logit
    
def getmyClassifier():
    net = Net2()
    net.load_state_dict(torch.load(WEIGHTS+'00059000_model.pth'))
    net = net.cuda()
    return net

In [16]:
class SEModule(nn.Module):

    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1,
                             padding=0)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1,
                             padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return module_input * x


class Bottleneck(nn.Module):
    """
    Base class for bottlenecks that implements `forward()` method.
    """
    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = self.se_module(out) + residual
        out = self.relu(out)

        return out


class SEBottleneck(Bottleneck):
    """
    Bottleneck for SENet154.
    """
    expansion = 4

    def __init__(self, inplanes, planes, groups, reduction, stride=1,
                 downsample=None):
        super(SEBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes * 2, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes * 2)
        self.conv2 = nn.Conv2d(planes * 2, planes * 4, kernel_size=3,
                               stride=stride, padding=1, groups=groups,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(planes * 4)
        self.conv3 = nn.Conv2d(planes * 4, planes * 4, kernel_size=1,
                               bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride
        
class SEResNetBottleneck(Bottleneck):
    """
    ResNet bottleneck with a Squeeze-and-Excitation module. It follows Caffe
    implementation and uses `stride=stride` in `conv1` and not in `conv2`
    (the latter is used in the torchvision implementation of ResNet).
    """
    expansion = 4

    def __init__(self, inplanes, planes, groups, reduction, stride=1,
                 downsample=None):
        super(SEResNetBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False,
                               stride=stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1,
                               groups=groups, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride

class SENet(nn.Module):

    def __init__(self, block, layers, groups, reduction, dropout_p=0.2,
                 inplanes=128, input_3x3=True, downsample_kernel_size=3,
                 downsample_padding=1, num_classes=1000):
        
        super(SENet, self).__init__()
        self.inplanes = inplanes
        if input_3x3:
            layer0_modules = [
                ('conv1', nn.Conv2d(3, 64, 3, stride=2, padding=1,
                                    bias=False)),
                ('bn1', nn.BatchNorm2d(64)),
                ('relu1', nn.ReLU(inplace=True)),
                ('conv2', nn.Conv2d(64, 64, 3, stride=1, padding=1,
                                    bias=False)),
                ('bn2', nn.BatchNorm2d(64)),
                ('relu2', nn.ReLU(inplace=True)),
                ('conv3', nn.Conv2d(64, inplanes, 3, stride=1, padding=1,
                                    bias=False)),
                ('bn3', nn.BatchNorm2d(inplanes)),
                ('relu3', nn.ReLU(inplace=True)),
            ]
        else:
            layer0_modules = [
                ('conv1', nn.Conv2d(3, inplanes, kernel_size=7, stride=2,
                                    padding=3, bias=False)),
                ('bn1', nn.BatchNorm2d(inplanes)),
                ('relu1', nn.ReLU(inplace=True)),
            ]
        layer0_modules.append(('pool', nn.MaxPool2d(3, stride=2,
                                                    ceil_mode=True)))
        self.layer0 = nn.Sequential(OrderedDict(layer0_modules))
        self.layer1 = self._make_layer(
            block,
            planes=64,
            blocks=layers[0],
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=1,
            downsample_padding=0
        )
        self.layer2 = self._make_layer(
            block,
            planes=128,
            blocks=layers[1],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.layer3 = self._make_layer(
            block,
            planes=256,
            blocks=layers[2],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.layer4 = self._make_layer(
            block,
            planes=512,
            blocks=layers[3],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.avg_pool = nn.AvgPool2d(7, stride=1)
        self.dropout = nn.Dropout(dropout_p) if dropout_p is not None else None
        self.last_linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, groups, reduction, stride=1,
                    downsample_kernel_size=1, downsample_padding=0):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=downsample_kernel_size, stride=stride,
                          padding=downsample_padding, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, groups, reduction, stride,
                            downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups, reduction))

        return nn.Sequential(*layers)

    def features(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def logits(self, x):
        x = self.avg_pool(x)
        if self.dropout is not None:
            x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

    def forward(self, x):
        x = self.features(x)
        x = self.logits(x)
        return x

class SeResnet50Classification(nn.Module):
    def __init__(self,num_class=4):
        super(SeResnet50Classification, self).__init__()
        e = SENet(SEResNetBottleneck, [3, 4, 6, 3], groups=1, reduction=16,
                  dropout_p=None, inplanes=64, input_3x3=False,
                  downsample_kernel_size=1, downsample_padding=0,
                  num_classes=1000)
        self.block = nn.ModuleList([
            e.layer0,
            e.layer1,
            e.layer2,
            e.layer3,
            e.layer4,
        ])
        e = None
        self.feature = nn.Conv2d(2048,32, kernel_size=1) #dummy conv for dim reduction
        self.logit = nn.Conv2d(32,num_class, kernel_size=1)

    def forward(self, x):
        batch_size,C,H,W = x.shape
        for i in range( len(self.block)):
            x = self.block[i](x)
        x = F.dropout(x,0.5,training=self.training)
        x = F.adaptive_avg_pool2d(x, 1)
        x = self.feature(x)
        logit = self.logit(x)
        return logit
    
def getSeresnet50Classifier():
    model = SeResnet50Classification()
    model=model.cuda()
    x = torch.load(WEIGHTS+'seresnet50_classifier.pth')
    model.load_state_dict(x)
    return model

In [17]:
class TestDataset(Dataset):
    '''Dataset for test prediction'''
    def __init__(self, root, df, transform):
        self.root = root
        df['ImageId'] = df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
        self.fnames = df['ImageId'].unique().tolist()
        self.num_samples = len(self.fnames)
        self.transform = transform

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = os.path.join(self.root, fname)
        image = cv2.imread(path)
        images = self.transform(image=image)["image"]
        return fname, images

    def __len__(self):
        return self.num_samples

In [18]:
sample_submission_path = '../input/severstal-steel-defect-detection/sample_submission.csv'
test_data_folder = "../input/severstal-steel-defect-detection/test_images"
df = pd.read_csv(sample_submission_path)
mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)

In [19]:
def getTransforms(tf):
    ds = []
    transform = albu.Compose([albu.Normalize(mean=mean,std=std),ToTensorV2()])
    ds.append(DataLoader(
        TestDataset(test_data_folder, df, transform),
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True
    ))
    for i in tf:
        transform = albu.Compose([i,albu.Normalize(mean=mean,std=std),ToTensorV2()])
        ds.append(DataLoader(
            TestDataset(test_data_folder, df, transform),
            batch_size=batch_size,
            shuffle=False,
            num_workers=0,
            pin_memory=True
        ))
    return ds

In [20]:
tf = [albu.HorizontalFlip(p=1)]#,albu.RandomContrast(p=1)]
ds = getTransforms(tf)

In [21]:
def sharpen(p,t=0.5):
        if t!=0:
            return p**t
        else:
            return p

In [22]:
def classifierPredict():
    
    res = []
    for _ in range(len(ds[0].dataset)*4):
        res.append([])
    itr = 0
    model = Model([getBibekClassifier(),getmyClassifier(),getSeresnet50Classifier()])
    total = len(ds[0].dataset)//batch_size
    for loaders_batch in tqdm(zip(*ds), total=total):
        preds = []
        for i in range(len(model.models)):
            preds.append([])
        image_file = []
        for i, (ids,features) in enumerate(loaders_batch):
            torch.cuda.empty_cache()
            p = model(features)
#             preds.append(p)
            for t in range(len(model.models)):
                preds[t].append(p[t])
#             if i == 1:
#                 p = torch.flip(p,[2])
#             if i == 2:
#                 p = torch.flip(p,[3])
#             if i!=1 and i!=2:
#                 p = sharpen(p,0)
#             else:
#                 p = sharpen(p)
#             preds.append(p)
            image_file = ids
        for t in range(len(preds)):
            preds[t] = torch.stack(preds[t])
            preds[t] = torch.mean(preds[t], dim=0)
            preds[t] = torch.sigmoid(preds[t])
        preds = torch.stack(preds)
        preds = torch.mean(preds, dim=0)
        preds = preds.detach().cpu().numpy()
        
        for p, file in zip(preds, image_file):
            file = os.path.basename(file)
            for i in range(4):
                p_channel = p[i]
                imageid_classid = file+'_'+str(i+1)
                p_channel = (p_channel>thresholds[i]).astype(np.uint8)
                if p_channel: 
                    res[itr] = {
                        'ImageId_ClassId': imageid_classid,
                        'EncodedPixels': '1 1'
                    }
                else:
                    res[itr] = {
                        'ImageId_ClassId': imageid_classid,
                        'EncodedPixels': ''
                    }
                itr += 1

    df = pd.DataFrame(res)
    del model
    del res
    return df

In [23]:
thresholds = [0.49, 0.45, 0.45, 0.49]
min_area = [600, 600, 1000, 2000]
from collections import OrderedDict
df_classification = classifierPredict()

113it [04:05,  2.18s/it]


In [24]:
import mxnet as mx
from mxnet.gluon import data, HybridBlock, nn
from mxnet.gluon.data.vision import transforms
from mxnet.gluon.model_zoo import vision
import glob
from mxnet import nd as F, gluon
from gluoncv import model_zoo as gm
from gluoncv.model_zoo.resnetv1b import resnet50_v1s, resnet101_v1s, resnet152_v1s
import mxnet as mx
from mxnet.gluon import nn
from mxnet.gluon.nn import HybridBlock


import mxnet as mx
from mxnet.gluon import nn
from mxnet.gluon.nn import HybridBlock

class ResNetBackbone(mx.gluon.HybridBlock):
    def __init__(self, backbone='resnet50', pretrained_base=True,dilated=True, **kwargs):
        super(ResNetBackbone, self).__init__()

        with self.name_scope():
            if backbone == 'resnet50':
                pretrained = resnet50_v1s(pretrained=pretrained_base, dilated=dilated, **kwargs)
            elif backbone == 'resnet101':
                pretrained = resnet101_v1s(pretrained=pretrained_base, dilated=dilated, **kwargs)
            elif backbone == 'resnet152':
                pretrained = resnet152_v1s(pretrained=pretrained_base, dilated=dilated, **kwargs)
            else:
                raise RuntimeError(f'unknown backbone: {backbone}')

            self.conv1 = pretrained.conv1
            self.bn1 = pretrained.bn1
            self.relu = pretrained.relu
            self.maxpool = pretrained.maxpool
            self.layer1 = pretrained.layer1
            self.layer2 = pretrained.layer2
            self.layer3 = pretrained.layer3
            self.layer4 = pretrained.layer4

    def hybrid_forward(self, F, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        c1 = self.layer1(x)
        c2 = self.layer2(c1)
        c3 = self.layer3(c2)
        c4 = self.layer4(c3)

        return c1, c2, c3, c4

class ResNetFPN(mx.gluon.HybridBlock):
    def __init__(self, backbone= 'resnet50', backbone_lr_mult=0.1, **kwargs):
        super(ResNetFPN, self).__init__()

        self.backbone_name = backbone
        self.backbone_lr_mult = backbone_lr_mult
        self._kwargs = kwargs

        with self.name_scope():
            self.backbone = ResNetBackbone(backbone=self.backbone_name, pretrained_base=False, dilated=False, **kwargs)

            self.head = _FPNHead(output_channels=256, **kwargs)

    def load_pretrained_weights(self):
        pretrained = ResNetBackbone(backbone=self.backbone_name, pretrained_base=True, dilated=False, **self._kwargs)
        backbone_params = self.backbone.collect_params()
        pretrained_weights = pretrained.collect_params()
        for k, v in pretrained_weights.items():
            param_name = backbone_params.prefix + k[len(pretrained_weights.prefix):]
            backbone_params[param_name].set_data(v.data())

        self.backbone.collect_params().setattr('lr_mult', self.backbone_lr_mult)

    def hybrid_forward(self,F, x):
        c1, c2, c3, c4 = self.backbone(x)
        p1, p2, p3, p4 = self.head(c1, c2, c3, c4)

        return p1, p2, p3, p4

class ResNetUnet(mx.gluon.HybridBlock):
    def __init__(self, backbone= 'resnet50', backbone_lr_mult=0.1, cls_branch=False, **kwargs):
        super(ResNetUnet, self).__init__()

        self.backbone_name = backbone
        self.backbone_lr_mult = backbone_lr_mult
        self.cls_branch = cls_branch
        self._kwargs = kwargs
        
        with self.name_scope():
            self.backbone = ResNetBackbone(backbone=self.backbone_name, pretrained_base=False, dilated=False, **kwargs)

            self.head = _UnetHead(**kwargs)

    def load_pretrained_weights(self):
        pretrained = ResNetBackbone(backbone=self.backbone_name, pretrained_base=True, dilated=False, **self._kwargs)
        backbone_params = self.backbone.collect_params()
        pretrained_weights = pretrained.collect_params()
        for k, v in pretrained_weights.items():
            param_name = backbone_params.prefix + k[len(pretrained_weights.prefix):]
            backbone_params[param_name].set_data(v.data())

        self.backbone.collect_params().setattr('lr_mult', self.backbone_lr_mult)

    def hybrid_forward(self,F, x):
        c1, c2, c3, c4 = self.backbone(x)
        out = self.head(c1, c2, c3, c4)
        if self.cls_branch:
            logits = F.max(F.mean(out, axis=1), axis=(1, 2))
            return out, logits
        return out

class _DecoderBlock(HybridBlock):
    def __init__(self, output_channels, norm_layer=nn.BatchNorm):
        super(_DecoderBlock, self).__init__()

        with self.name_scope():
            self.block = nn.HybridSequential()
            self.block.add(ConvBlock(output_channels, kernel_size=3, padding=1, norm_layer=norm_layer))
            self.block.add(ConvBlock(output_channels, kernel_size=3, padding=1, norm_layer=norm_layer))

    def hybrid_forward(self, F, x, y=None):

        if y is not None:
            x = F.contrib.BilinearResize2D(x, scale_height=2, scale_width=2)
            x = F.concat(x, y, dim=1)
        out = self.block(x)

        return out


class _UnetHead(HybridBlock):
    def __init__(self, num_classes, output_channels=[256, 128, 64, 32], scale=4, norm_layer=nn.BatchNorm):
        super(_UnetHead, self).__init__()
        
        self.scale = scale
        with self.name_scope():
            self.block4 = _DecoderBlock(output_channels[0], norm_layer=norm_layer)
            self.block3 = _DecoderBlock(output_channels[1], norm_layer=norm_layer)
            self.block2 = _DecoderBlock(output_channels[2], norm_layer=norm_layer)
            self.block1 = _DecoderBlock(output_channels[3], norm_layer=norm_layer)
            self.postprocess_block = nn.Conv2D(num_classes, kernel_size=1)

    def hybrid_forward(self, F, c1, c2, c3, c4):

        p4 = self.block4(c4)
        p3 = self.block3(p4, c3)
        p2 = self.block2(p3, c2)
        p1 = self.block1(p2, c1)
        if self.scale > 1:
            p1 = F.contrib.BilinearResize2D(p1, scale_height=self.scale, scale_width=self.scale)
        out = self.postprocess_block(p1)

        return out


class _FPNHead(HybridBlock):
    def __init__(self, output_channels=256, norm_layer=nn.BatchNorm):
        super(_FPNHead, self).__init__()
        self._hdsize = {}

        with self.name_scope():
            self.block4 = ConvBlock(output_channels, kernel_size=1, norm_layer=norm_layer)
            self.block3 = ConvBlock(output_channels, kernel_size=1, norm_layer=norm_layer)
            self.block2 = ConvBlock(output_channels, kernel_size=1, norm_layer=norm_layer)
            self.block1 = ConvBlock(output_channels, kernel_size=1, norm_layer=norm_layer)

    def hybrid_forward(self, F, c1, c2, c3, c4):
        p4 = self.block4(c4)
        p3 = self._resize_as(F, 'id_1', p4, c3) + self.block3(c3)
        p2 = self._resize_as(F, 'id_2', p3, c2) + self.block2(c2)
        p1 = self._resize_as(F, 'id_3', p2, c1) + self.block1(c1)

        return p1, p2, p3, p4

    def _resize_as(self, F, name, x, y):
        h_key = name + '_h'
        w_key = name + '_w'

        if hasattr(y, 'shape'):
            _, _, h, w = y.shape
            _, _, h2, w2 = x.shape

            if h == h2 and w == w2:
                h = 0
                w = 0

            self._hdsize[h_key] = h
            self._hdsize[w_key] = w
        else:
            h, w = self._hdsize[h_key], self._hdsize[w_key]

        if h == 0 and w == 0:
            return x
        else:
            return F.contrib.BilinearResize2D(x, height=h, width=w)


class SemanticFPNHead(HybridBlock):
    def __init__(self, num_classes, output_channels=128, norm_layer=nn.BatchNorm):
        super(SemanticFPNHead, self).__init__()
        self._hdsize = {}

        with self.name_scope():
            self.block4_1 = ConvBlock(output_channels, kernel_size=3, padding=1, norm_layer=norm_layer)
            self.block4_2 = ConvBlock(output_channels, kernel_size=3, padding=1, norm_layer=norm_layer)
            self.block4_3 = ConvBlock(output_channels, kernel_size=3, padding=1, norm_layer=norm_layer)

            self.block3_1 = ConvBlock(output_channels, kernel_size=3, padding=1, norm_layer=norm_layer)
            self.block3_2 = ConvBlock(output_channels, kernel_size=3, padding=1, norm_layer=norm_layer)

            self.block2 = ConvBlock(output_channels, kernel_size=3, padding=1, norm_layer=norm_layer)
            self.block1 = ConvBlock(output_channels, kernel_size=1, norm_layer=norm_layer)

            self.postprocess_block = nn.Conv2D(num_classes, kernel_size=1)

    def hybrid_forward(self, F, c1, c2, c3, c4):
        out4 = self._resize_as(F, 'id_1', self.block4_1(c4), c3)
        out4 = self._resize_as(F, 'id_2', self.block4_2(out4), c2)
        out4 = self._resize_as(F, 'id_3', self.block4_3(out4), c1)

        out3 = self._resize_as(F, 'id_4', self.block3_1(c3), c2)
        out3 = self._resize_as(F, 'id_5', self.block3_2(out3), c1)

        out2 = self._resize_as(F, 'id_6', self.block2(c2), c1)

        out1 = self.block1(c1)

        out = out1 + out2 + out3 + out4

        out = self.postprocess_block(out)
        out = F.contrib.BilinearResize2D(out,scale_height=4,scale_width=4)
        return out

    def _resize_as(self, F,name, x, y):
        h_key = name + '_h'
        w_key = name + '_w'

        if hasattr(y, 'shape'):
            _, _, h, w = y.shape
            _, _, h2, w2 = x.shape

            if h == h2 and w == w2:
                h = 0
                w = 0

            self._hdsize[h_key]=h
            self._hdsize[w_key]=w
        else:
            h, w = self._hdsize[h_key], self._hdsize[w_key]

        if h == 0 and w == 0:
            return x
        else:
            return F.contrib.BilinearResize2D(x,height=h,width=w)


class ConvBlock(HybridBlock):
    def __init__(self, output_channels, kernel_size, padding=0, activation='relu', norm_layer=nn.BatchNorm):
        super().__init__()
        self.body = nn.HybridSequential()
        self.body.add(
            nn.Conv2D(output_channels, kernel_size=kernel_size, padding=padding, activation=activation),
            norm_layer(in_channels=output_channels)
        )

    def hybrid_forward(self, F, x):
        return self.body(x)

class SteelUnet(HybridBlock):
    
    def __init__(self, n_classes=5, ctx=mx.cpu()):
        super().__init__()
        with self.name_scope():
            self.feature_extractor = ResNetFPN()
            self.segment_head = SemanticFPNHead(num_classes=n_classes)
    def hybrid_forward(self, F, x):
        fpn_feature = self.feature_extractor(x)
        segment_out = self.segment_head(*fpn_feature)
        return segment_out

In [25]:
ctx = mx.gpu()
unet = ResNetUnet(output_channels=[256, 128, 64, 32], num_classes=5)
unet.load_parameters('../input/mxnetweights/unet_34_-1.params')
unet.collect_params().reset_ctx(ctx)

In [26]:
def remove_small_one(predict, min_size):
    H,W = predict.shape
    num_component, component = cv2.connectedComponents(predict.astype(np.uint8))
    predict = np.zeros((H,W), np.bool)
    for c in range(1,num_component):
        p = (component==c)
        if p.sum()>min_size:
            predict[p] = True
    return predict

In [27]:
# import random
# import time
# test_stage
trans = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=(0.485, 0.456, 0.406),
                   std=(0.229, 0.224, 0.225)
                )
            ]
        )

# test_dir = "../input/severstal-steel-defect-detection/test_images/"

# imglists = glob.glob(test_dir + '/*g')
# oriims = []
# preds = []
# random.shuffle(imglists)
# ImageId_ClassIds = []
# EncodedPixels = []

augs = ['flip_lr', 'flip_ud']
# thresholds = [0.5, 0.5, 0.5, 0.5]
# min_area = [600, 600, 1000, 2000]
# min_area = [1, 1, 1, 1]
# t1 = time.time()
# for i, item in enumerate(tqdm(imglists)):
#     timg = cv2.imread(item)[:, :, ::-1]
#     img = mx.nd.array([timg])
#     input_img = trans(img)
#     num_aug = 0
    
#     if 1:
#         out = unet(input_img.as_in_context(ctx))
        
#         out = F.softmax(out, axis=1)
#         out = F.where(out > 0.5, out, F.zeros_like(out))
#         pred_inds = F.argmax(out, axis=1)
#         oriims.append(timg)
#         preds.append(pred_inds)
#         out_mask = sharpen(out, 0)
        
#         num_aug += 1
#     if 'flip_lr' in augs:
#         input_img_lr = F.flip(input_img, axis=3)
#         out = unet(input_img_lr.as_in_context(ctx))
#         out = F.softmax(out, axis=1)
#         out = F.where(out > 0.5, out, F.zeros_like(out))
#         out_mask += sharpen(F.flip(out, axis=3))
#         num_aug += 1
    
#     out_mask = out_mask * 1.0 / num_aug
#     out = out_mask[:, 1:, :, :].asnumpy()
#     ImageId = item.split('/')[-1]
#     pred_inds = pred_inds.asnumpy()
#     for j in range(4):
#         Id = ImageId + '_'+str(j+1)
#         tmp_mask = np.where(out[:, j, :, :] > thresholds[j], 1.0, 0)
# #         tmp_mask = np.where(pred_inds[0, :, :]==(j+1), 1, 0)
#         if np.sum(tmp_mask) < min_area[j]:
#             tmp_mask = np.zeros_like(tmp_mask)
#         if np.sum(tmp_mask) < 10:
#             EncodedPixel = ''
#         else:
#             EncodedPixel = mask2rle(tmp_mask)

#         ImageId_ClassIds.append(Id)
#         EncodedPixels.append(EncodedPixel)
# dur = time.time() - t1
# print("cost time:{}".format(dur))

In [28]:
# submission =  pd.read_csv("../input/severstal-steel-defect-detection/sample_submission.csv")
# print(len(ImageId),len(submission['ImageId_ClassId']))
# # len(set(submission['ImageId_ClassId'])-set(Ids))
# # assert set(Ids) == set(submission['ImageId_ClassId'])

# for i, encoded in zip(ImageId_ClassIds,EncodedPixels):
#     submission.loc[submission['ImageId_ClassId']==i,["EncodedPixels"]] =  encoded

# submission.to_csv('submission.csv',index=False)

In [29]:
def getMXPred(item):
    timg = cv2.imread(item)[:, :, ::-1]
    img = mx.nd.array([timg])
    input_img = trans(img)
#     num_aug = 0
    
    if 1:
        out = unet(input_img.as_in_context(ctx))
        out = F.softmax(out, axis=1)
        out = F.where(out > 0.5, out, F.zeros_like(out))
#         pred_inds = F.argmax(out, axis=1)
        out_mask = sharpen(out, 0)
#         num_aug += 1
        
#     if 'flip_lr' in augs:
#     if i == 1:
#         input_img_lr = F.flip(input_img, axis=3)
#         out = unet(input_img.as_in_context(ctx))
#         out = F.flip(out, axis=3)
#         out = F.softmax(out, axis=1)
#         out = F.where(out > 0.5, out, F.zeros_like(out))
#         out_mask = sharpen(F.flip(out, axis=3))
#         num_aug += 1
    
#     out = out * 1.0 / 2
    out = out_mask[:, 1:, :, :].asnumpy()
    return out   

In [30]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [31]:
thresholds = [0.45, 0.49, 0.45, 0.49]
min_area = [600, 600, 1000, 2000]

res = []
# Iterate over all TTA loaders
total = len(datasets[0])//batch_size
for loaders_batch in tqdm_notebook(zip(*loaders), total=total):
    preds = []
    for m in range(len(model.models)):
        preds.append([])
    image_file = []
    
    for i, batch in enumerate(loaders_batch):
        image_file = batch['image_file']
        features = batch['features'].cuda()
        x = model(features)
        for t,v in enumerate(x):
            p = datasets[i].inverse(v)
            preds[t].append(p)
#         if i == 0:
#             p = sharpen(p,0)
#         else:
#             p = sharpen(p,0.5)
        # inverse operations for TTA
#         p = p.detach().cpu().numpy()
#         preds.append(p)
    # TTA mean
    for t in range(len(model.models)):
        preds[t] = torch.stack(preds[t])
        preds[t] = torch.mean(preds[t],dim=0)
        preds[t] = torch.sigmoid(preds[t])
        
    preds = torch.stack(preds)
    preds = torch.mean(preds, dim=0)
    x = []
    for f in image_file:
        out = getMXPred(f)
        x.append(out)
    x = np.concatenate(x)
    x = torch.FloatTensor(x).to('cuda')
        
    preds = torch.stack([preds,x])
    preds = torch.mean(preds, dim=0)
    preds = preds.detach().cpu().numpy()
    # Batch post processing
    for p, file in zip(preds, image_file):
        file = os.path.basename(file)
        # Image postprocessing
        for i in range(4):
            p_channel = p[i]
            imageid_classid = file+'_'+str(i+1)
            p_channel = (p_channel>thresholds[i]).astype(np.uint8)
            if p_channel.sum() < min_area[i]:
                p_channel = np.zeros(p_channel.shape, dtype=p_channel.dtype)
#             if np.sum(p_channel) < 10:
#                 res.append({
#                     'ImageId_ClassId': imageid_classid,
#                     'EncodedPixels': ''
#                 })
#             else:
            tp = df_classification[df_classification['ImageId_ClassId']==imageid_classid]['EncodedPixels'].values
            mask = mask2rle(p_channel)
            if tp[0] == '1 1' and mask != '': 
#                 if mask !='':
                res.append({
                    'ImageId_ClassId': imageid_classid,
                    'EncodedPixels': mask
                })
            else:
                res.append({
                    'ImageId_ClassId': imageid_classid,
                    'EncodedPixels': ''
                })
                
df = pd.DataFrame(res)
df.to_csv('submission.csv', index=False)

In [32]:
# def getMXPred(item,i):
#     timg = cv2.imread(item)[:, :, ::-1]
#     img = mx.nd.array([timg])
#     input_img = trans(img)
#     num_aug = 0
    
#     if i==0:
#         out = unet(input_img.as_in_context(ctx))
# #         out = F.softmax(out, axis=1)
# #         out = F.where(out > 0.5, out, F.zeros_like(out))
# #         pred_inds = F.argmax(out, axis=1)
# #         out_mask = sharpen(out, 0)
# #         num_aug += 1
        
#     if i==1:
#         input_img_lr = F.flip(input_img, axis=3)
#         out = unet(input_img_lr.as_in_context(ctx))
#         out = F.flip(out, axis=3)
# #         print(1)
# #         out = F.softmax(out, axis=1)
# #         out = F.where(out > 0.5, out, F.zeros_like(out))
# #         out_mask += sharpen(F.flip(out, axis=3))
# #         num_aug += 1
    
# #     out_mask = out_mask * 1.0 / num_aug
#     out = out[:, 1:, :, :].asnumpy()
#     return out   

In [33]:
# res = []
# # Iterate over all TTA loaders
# total = len(datasets[0])//batch_size
# for loaders_batch in tqdm_notebook(zip(*loaders), total=total):
#     preds = []
#     image_file = []
#     for i, batch in enumerate(loaders_batch):
#         image_file = batch['image_file']
#         features = batch['features'].cuda()
#         resx = []
#         x = []
#         for f in image_file:
#             out = getMXPred(f,i)
#             if i == 0:
#                 out = sharpen(out,0)
#             else:
#                 out = sharpen(out,0.5)
#             x.append(out)
#         x = np.concatenate(x)
#         x = torch.FloatTensor(x).to('cuda')
#         x = torch.sigmoid(x)
#         resx.append(x)
#         with torch.no_grad():
#             for m in [unet_se_resnext50_32x4d, unet_mobilenet2, unet_resnet34]:
#                 x = m(features)
#                 x = datasets[i].inverse(x)
#                 x = torch.sigmoid(x)
                
#                 if i == 0:
#                     x = sharpen(x,0)
#                 else:
#                     x = sharpen(x,0.5)
#                 resx.append(x)
        
            
#         resx = torch.stack(resx)
#         resx = torch.mean(resx, dim=0)
# #         resx = torch.sigmoid(resx)
        
#         # inverse operations for TTA
# #         p = p.detach().cpu().numpy()
#         preds.append(resx)
#     # TTA mean
#     preds = torch.stack(preds)
#     preds = torch.mean(preds, dim=0)
#     preds = preds.detach().cpu().numpy()
#     # Batch post processing
#     for p, file in zip(preds, image_file):
#         file = os.path.basename(file)
#         # Image postprocessing
#         for i in range(4):
#             p_channel = p[i]
#             imageid_classid = file+'_'+str(i+1)
#             p_channel = (p_channel>thresholds[i]).astype(np.uint8)
#             if p_channel.sum() < min_area[i]:
#                 p_channel = np.zeros(p_channel.shape, dtype=p_channel.dtype)
# #             if np.sum(p_channel) < 10:
# #                 res.append({
# #                     'ImageId_ClassId': imageid_classid,
# #                     'EncodedPixels': ''
# #                 })
# #             else:
#             tp = df_classification[df_classification['ImageId_ClassId']==imageid_classid]['EncodedPixels'].values
#             mask = mask2rle(p_channel)
#             if tp[0] == '1 1' and mask != '': 
# #                 if mask !='':
#                 res.append({
#                     'ImageId_ClassId': imageid_classid,
#                     'EncodedPixels': mask
#                 })
#             else:
#                 res.append({
#                     'ImageId_ClassId': imageid_classid,
#                     'EncodedPixels': ''
#                 })
                
# df = pd.DataFrame(res)
# df.to_csv('submission.csv', index=False)

In [34]:
# df = pd.DataFrame(res)
# df = df.fillna('')
# df.to_csv('submission.csv', index=False)

In [35]:
# df.head(15)

In [36]:
def summarise_submission_csv(df):
    text = ''
    df['Class'] = df['ImageId_ClassId'].str[-1].astype(np.int32)
    df['Label'] = (df['EncodedPixels']!='').astype(np.int32)
    num_image = len(df)//4
    num = len(df)

    pos = (df['Label']==1).sum()
    neg = num-pos


    pos1 = ((df['Class']==1) & (df['Label']==1)).sum()
    pos2 = ((df['Class']==2) & (df['Label']==1)).sum()
    pos3 = ((df['Class']==3) & (df['Label']==1)).sum()
    pos4 = ((df['Class']==4) & (df['Label']==1)).sum()

    neg1 = num_image-pos1
    neg2 = num_image-pos2
    neg3 = num_image-pos3
    neg4 = num_image-pos4


    text += 'compare with LB probing ... \n'
    text += '\t\tnum_image = %5d(1801) \n'%num_image
    text += '\t\tnum  = %5d(7204) \n'%num
    text += '\n'

    text += '\t\tpos1 = %5d( 128)  %0.3f\n'%(pos1,pos1/128)
    text += '\t\tpos2 = %5d(  43)  %0.3f\n'%(pos2,pos2/43)
    text += '\t\tpos3 = %5d( 741)  %0.3f\n'%(pos3,pos3/741)
    text += '\t\tpos4 = %5d( 120)  %0.3f\n'%(pos4,pos4/120)
    text += '\n'

    text += '\t\tneg1 = %5d(1673)  %0.3f  %3d\n'%(neg1,neg1/1673, neg1-1673)
    text += '\t\tneg2 = %5d(1758)  %0.3f  %3d\n'%(neg2,neg2/1758, neg2-1758)
    text += '\t\tneg3 = %5d(1060)  %0.3f  %3d\n'%(neg3,neg3/1060, neg3-1060)
    text += '\t\tneg4 = %5d(1681)  %0.3f  %3d\n'%(neg4,neg4/1681, neg4-1681)
    text += '--------------------------------------------------\n'
    text += '\t\tneg  = %5d(6172)  %0.3f  %3d \n'%(neg,neg/6172, neg-6172)
    text += '\n'

#     if 1:
#     #compare with reference
#         pass

    return text

In [37]:
text = summarise_submission_csv(df_classification)
print(text) 

compare with LB probing ... 
		num_image =  1801(1801) 
		num  =  7204(7204) 

		pos1 =   123( 128)  0.961
		pos2 =    14(  43)  0.326
		pos3 =   644( 741)  0.869
		pos4 =   116( 120)  0.967

		neg1 =  1678(1673)  1.003    5
		neg2 =  1787(1758)  1.016   29
		neg3 =  1157(1060)  1.092   97
		neg4 =  1685(1681)  1.002    4
--------------------------------------------------
		neg  =  6307(6172)  1.022  135 


